In [ ]:
def list_rule(*rule_spec):
    case rule_spec:
        match (rule is str, *args, {**kwds}):
            print(1, rule, args, kwds)
        match (rule is str, *args):
            print(2, rule, args, {})
    else:
        raise Exception("Invalid list-rule syntax: {}".format(rule_spec))

list_rule('req',2,3, {1:2})
list_rule('req',5,'h')
list_rule('req')

In [ ]:
def dict_rule(**rule_spec):
    case rule_spec:
        match {'rule': rule is str}:
            print(1, rule, [], {})
        match {'rule': rule is str, 'args': [*args]}:
            print(21, rule, args, {})
        match {'rule': rule is str, 'kwds': {**kwds}}:
            print(22, rule, [], kwds)
        match {'rule': rule is str, 'args': [*args], 'kwds': {**kwds}}:
            print(3, rule, args, kwds)
    else:
        raise Exception("Invalid dict-rule syntax: {}".format(rule_spec))

dict_rule(rule='rgex')
dict_rule(rule='rgex', args=[1])
dict_rule(rule='rgex', args=[1], kwds={1:1})
dict_rule(rule='rgex', kwds={1:1})

In [ ]:
def rule(rule_spec):
    case rule_spec:
        match r is str:
            print(10, r, [], {})
        match [*r]:
            list_rule(*r)
        match {**r}:
            dict_rule(*r)
    else:
        raise Exception("Invalid rule syntax: {}".format(rule_spec))
    return  

rule(['ggg'])
rule({'func': 'ggg'})
rule('func')

In [ ]:
def parse_rule_spec(rule_spec, no_recurse=False):
    case rule_spec:
        match rule is str:
            #print(0, rule, [], {})
            return (rule, [], {})

        match (rule is str, [*args] is (list, tuple), {**kwds}):
            #print(10, rule, args, kwds)
            return (rule, args, kwds)
        match (rule is str, [*args] is (list, tuple)):
            #print(20, rule, args, {})
            return (rule, args, {})
        match (rule is str, ):
            #print(30, rule, [], {})
            return (rule, [], {})

        match {'rule': rule is str}:
            #print(110, rule, [], {})
            return (rule, [], {})
        match {'rule': rule is str, 'args': [*args] is (list, tuple)}:
            #print(121, rule, args, {})
            return (rule, args, {})
        match {'rule': rule is str, 'kwds': {**kwds}}:
            #print(122, rule, [], kwds)
            return (rule, [], kwds)
        match {'rule': rule is str, 'args': [*args] is (list, tuple), 'kwds': {**kwds}}:
            #print(130, rule, args, kwds)
            return (rule, args, kwds)

        ## Attempt parsing them as multi-rules,
        #  but descend only 1 level (to protect config-user).
        #
        match rule_specs is (list, tuple) if not no_recurse:
            return [parse_rule_spec(r, no_recurse=True) 
                    for r in rule_specs]
            
    else:
        raise Exception("Invalid rule syntax: {}".format(rule_spec))

assert parse_rule_spec(['req', [2,3], {1:2}]) ==  ('req', [2, 3], {1: 2})
assert parse_rule_spec(['req', [5,'h']]) ==  ('req', [5, 'h'], {})
assert parse_rule_spec(['req']) ==  ('req', [], {})


assert parse_rule_spec({'rule': 'rgex'}) ==  ('rgex', [], {})
assert parse_rule_spec({'rule': 'rgex', 'args': [1]}) == ('rgex', [1], {})
assert parse_rule_spec({'rule': 'rgex', 'args': [1], 'kwds': {1:1}}) == ('rgex', [1], {1: 1})
assert parse_rule_spec({'rule': 'rgex', 'kwds': {1:1}}) == ('rgex', [], {1: 1})

assert parse_rule_spec('frag') ==  ('frag', [], {})

assert parse_rule_spec(['f1', 'f2', ['f3']]) == [('f1', [], {}), ('f2', [], {}), ('f3', [], {})]

## ERRORS
#parse_rule_spec(['g', 'jdd', 'h'])
#parse_rule_spec(['f1', 'f2', [['f3']]])
